<a href="https://colab.research.google.com/github/DaSilva-JV/AIED2025/blob/main/Allocating_Dynamic_and_Finite_Resources_to_a_set_of_Tasks_uu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Google Drive to Collaboratory
drive.mount('/content/gdrive')

# Install pulp
!pip install pulp

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 54.1 MB/s eta 0:00:00


In [ ]:
test_type = '1'
test_name = {'1': 'mt', '2': 'cn', '3': 'ch', '4': 'lc'}
directory_name = {'1': 'Matemática', '2': 'Ciências da Natureza', '3': 'Ciências Humanas', '4': 'Linguagens e Códigos'}


# Getting the tasks k-parameter
df = pd.read_csv('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/' + directory_name[test_type] + '/calculated_k_' + test_name[test_type] + '.csv')
k = np.array(df['k_parameter'])

# Getting the number of items solved for each volunteer (for all the 10.000)
df = pd.read_csv('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/' + directory_name[test_type] + '/n_correct_answers_' + test_name[test_type] + '.csv')
n_solved = np.array(df['n_solved'])

# Getting the sampled volunteers
df = pd.read_csv('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/Sampled_Volunteers.csv')
sampled_volunteers = np.array(df['Volunteer'])

# Getting the volunteers parameters (in the case of UU, we do not have any information about thetas)
#df = pd.read_csv('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/' + directory_name[test_type] + '/Estimated_Theta_' + test_name[test_type] + '.csv')
#all_theta = np.array(df['Theta'])

# Load the 180 answers from the 10000 students
df_respostas = pd.read_csv('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/file.txt', sep=' ', header=None)
df_respostas = df_respostas.iloc[:, 0:175]

if test_type == '1':
    df_resp = df_respostas.iloc[:, 0:45]
elif test_type == '2':
    df_resp = df_respostas.iloc[:, 45:90]
elif test_type == '3':
    df_resp = df_respostas.iloc[:, 90:135]
else:
    df_resp = df_respostas.iloc[:, 135:175]


#print(df_resp)


'''
# Load the parameters a, b and c for the 45 or 40 items
df_param = pd.read_fwf('/content/gdrive/MyDrive/Allocating Dynamic and Finite Resources to a set of known Tasks/' + directory_name[test_type] + '/param_enem_' + test_name[test_type] + '.txt')

# Add the type of the item
df_param['Type'] = test_type

# Put all parameters together
df_items = df_param

# Delete the column Unnamed
del df_items['Unnamed: 0']
'''


# Prior Mean and STD Deviation on log(a) [a follows a lognormal distribution]
pm_alpha = 0.0
pv_alpha = 0.5
pv_alpha = pv_alpha**2


# Prior Mean and STD Deviation on b [b follows a normal distribution]
pm_b = 0.0
pv_b = 1
pv_b = pv_b**2


# Prior Mean and STD Deviation on c [c follows a beta distribution]
# For these values, p = 0.2. For values ​​(5, 23); we have p = 0.15
# This value would be the shot hit value
alp_c = 5
bet_c = 17


# Values ​​for theta ranges
x = [-4.0000, -3.1111, -2.2222, -1.3333, -0.4444, 0.4444, 1.3333, 2.2222, 3.1111, 4.0000]
ak = [0.000119, 0.002805, 0.03002, 0.1458, 0.3213, 0.3213, 0.1458, 0.03002, 0.002805, 0.000119]



In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import time
import pulp
import math
from scipy.stats import norm
import csv
from datetime import datetime
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
import gspread_dataframe as gd


# ------------------------------------------------------------------------------
# IRT (Three-Parameter Logistic Model) model
def prob_3pl(theta_, a_, b_, c_):
    d = 1
    return c_ + (1 - c_) * (1.0 / (1.0 + np.exp(d * a_ * (b_ - theta_))))


# Modified EAP to update the theta estimate with each submission
def expected_a_posteriori_mod(item, a, b, c, ans_presented, probs_, num_, presented):
    # A priori distribution of theta (normal with mean 0 and std 1)
    mu = 0
    var = 1
    theta_b = np.linspace(-4, 4, num=num_)
    blf0 = norm.pdf(theta_b, mu, var)
    blf0 = blf0 / sum(blf0)

    # Calculated the probabilities using the IRT-3PL
    probs_[item] = prob_3pl(theta_b, a, b, c)

    # Update the blf with the Likelihood from the answers
    blf = blf0

    # print(presented)
    # print(ans_presented)

    for i, ans in enumerate(presented):  # for tasks already presented only
        if ans_presented[i]:
            blf = blf * probs_[ans, ]
        else:
            blf = blf * (1 - probs_[ans, ])
        # Normalize after updated for all volunteers
        blf = blf / sum(blf)

    # theta_eap.append(sum(blf * theta_b))
    theta_eap = sum(blf * theta_b)  # Calculating the average

    # calculate cumulative sum

    # Calculation of the Posterior Standard Deviation
    psd = math.sqrt(sum(blf * (theta_b - theta_eap) ** 2))

    return theta_eap, psd
# ------------------------------------------------------------------------------



# ------------------------------------------------------------------------------
# IRT (Three-Parameter Logistic Model)
def prob_irt_3pl(task_volunteer, a, b, c, theta):
    i = task_volunteer[0]  # Task
    j = task_volunteer[1]  # Volunteer
    d = 1
    return c[i] + (1 - c[i]) * (1.0 / (1.0 + math.exp(d * a[i] * (b[i] - theta[j]))))



def linear_programming_mod(n_items, n_volunteers, a, b, c, theta, n_tries, n_solutions, vezes_resolvidas, ep):
    # Create the 'prob' variable to contain the problem data
    prob = pulp.LpProblem("ItemsVolunteers", pulp.LpMaximize)

    # create list of all possible combinations of questions and volunteers
    possible_item_volunteer = [(i, j) for i in range(n_items) for j in range(n_volunteers)]

    # create a binary variable to state that a question was presented to a volunteer
    x = pulp.LpVariable.dicts("item_volunteer", possible_item_volunteer, lowBound=0, upBound=1, cat=pulp.LpContinuous)

    # The objective function is added to 'prob' first
    prob += pulp.lpSum([prob_irt_3pl(t_v, a, b, c, theta) * x[t_v] for t_v in possible_item_volunteer])

    # ----------- #
    # Constraints #
    # ----------- #
    # specify the number of tries for each volunteer
    for j in range(n_volunteers):
        prob += pulp.lpSum([x[(i, j)] for i in range(n_items)]) <= n_tries[j]

    # specify the desired quantity of solutions for each question
    for i in range(n_items):
        if (n_solutions[i] - vezes_resolvidas[i]) > 0:
            prob += pulp.lpSum([prob_irt_3pl((i, j), a, b, c, theta) * x[(i, j)] for j in range(n_volunteers)]) <= (n_solutions[i] - vezes_resolvidas[i])
        else:
            prob += pulp.lpSum([prob_irt_3pl((i, j), a, b, c, theta) * x[(i, j)] for j in range(n_volunteers)]) == 0

    # The problem data is written to an .lp file
    prob.writeLP("ItemsVolunteers.lp")

    pulp.LpSolverDefault.msg = 1

    # The problem is solved using PuLP's choice of Solver
    prob.solve()
    # prob.solve(pulp.PULP_CBC_CMD(gapRel=0.1))
    # status = prob.solve(solver=pulp.GLPK(msg=False))

    # Write the solutions in a pandas dataframe
    df = pd.DataFrame(columns=['item', 'volunteer', 'action'])
    # Each of the variables is printed with it's resolved optimum value
    for v in prob.variables():
        s = v.name.replace('item_volunteer_(', '').replace('_', '').replace(')', '')  # Leaves only the value of the item and the volunteer separated by a comma in a string
        s_list = s.split(',')  # Transforms the string into a list

        # Transforms string format to int in each list component
        s_list[0] = int(s_list[0])
        s_list[1] = int(s_list[1])

        # Add the PL solution (corresponding to whether or not the item should be presented to the volunteer)
        s_list.append(v.varValue)

        # Add the solution to a dataframe
        if s_list[2] > 0:
            df = df.append(dict(zip(df.columns, s_list)), ignore_index=True)

    #print('#------------------------------#')
    #print("Total number of solutions = ", pulp.value(prob.objective))  # The optimised objective function value is printed to the screen
    #print("Status:", pulp.LpStatus[prob.status])  # The status of the solution is printed to the screen
    #print("Episódio {}".format(ep))  # The ongoing/finished episode
    #print('#------------------------------#')

    return df
# ------------------------------------------------------------------------------



# Policy for choosing the tasks to volunteers
def pl_policy(vol_, df_sol):
    # Take the questions found from the IntegerProgramming
    questions = df_sol[df_sol['volunteer'] == vol_]
    w = list(questions['action'])

    valid_questions = {'item': [], 'action_w': []}
    # Checks if the task has not yet been presented to the volunteer and if it has not yet been solved the desired number of times
    for j, q in enumerate(questions['item']):
        q = int(q)
        if (q not in tarefas_apresentadas[vol_]) and (vezes_resolvidas[q] < n_solutions[q]):
            valid_questions['item'].append(q)
            valid_questions['action_w'].append(w[j])

    # Checks if there are tasks available for the user
    if valid_questions['item']:
        # if it exists, selects a task at random and adds the task to the list presented to the user
        probs = np.array(valid_questions['action_w']) / sum(valid_questions['action_w'])
        tarefa = np.random.choice(valid_questions['item'], p=probs)
        return tarefa
    else:
        # The first available task is returned even if it was not selected by LP
        for q in range(n_questions):
            if (q not in tarefas_apresentadas[vol_]) and (vezes_resolvidas[q] < n_solutions[q]):
                return q
        return -1


# Policy for choosing randomly tasks to volunteers
def random_task(volunteer):
    valid_questions = list()
    for q in range(n_questions):
        # Checks if the task has not yet been presented to the volunteer and if it has not yet been solved the desired number of times
        if (q not in tarefas_apresentadas[volunteer]) and (vezes_resolvidas[q] < n_solutions[q]):
            valid_questions.append(q)

    # Checks if there are tasks available for the user
    if valid_questions:
        # if it exists, selects a task at random and adds the task to the list presented to the user
        tarefa = np.random.choice(valid_questions)
        return tarefa
    else:
        return -1




# ------------------------------------------------------------------------------


# ------------------------------------------------------------------------------




# Program to implement marginalized bayesian item parameter estimation
# B = difficulty
# A = discrimination
# C = guessing
def bayesian_item_parameter_estimation(pl, lxk, rik, fik, u_li, item, vol, a_i, b_i, c_i):

    # Item Scores Pattern Frequencies
    # fpt(l) is the number of resources in each response pattern of the T tasks
    fpt = np.ones(n_volunteers)

    # Number of EM algorithm cycles (E-step and M-step cycles)
    mnc = 1

    for nc in range(mnc):
        #print("Ciclo = {}".format(nc))


        # E-step: get expected R and N
        #for l in range(n_volunteers):
        for k in range(10):
            #pq = []
            #for i in range(n_questions):

            # Calculations of P*(x), Q*(x), P(x) and Q(x)
            dev = - a_i*(x[k] - b_i)
            p_star = 1/(1 + math.exp(dev))
            q_star = 1 - p_star

            p = c_i + (1 - c_i)*p_star
            q = 1 - p

            '''
            # Get the result of the answer for pattern l and item i
            u_li = df_volunts[df_volunts["Padrão Resp"] == vol]["Item " + str(item)]
            if u_li.count() == 0:  # Checks if the result is empty
                continue
            else:
                u_li = u_li.iloc[0]
            '''

            if u_li == 1:
                pq = p  # The probability of success
            else:
                pq = q  # The probability of error (1-p)

            try:
                lxk[vol,k] = lxk[vol,k] * pq
            except:
                print("Vol = {}".format(vol))
                print("Item = {}".format(item))
                print("LXK shape = {}".format(lxk.shape))
                print("PQ length = {}".format(len(pq)))
                raise("Erro!!")

        pl[vol] = 0
        for k in range(10):
            pl[vol] = pl[vol] + lxk[vol,k] * ak[k]


        #for i in range(n_questions):
        for k in range(10):
        #for l in range(2**n_questions):

            '''
            # Get the result of the answer for pattern l and item i
            u_li = df_volunts[df_volunts["Padrão Resp"] == vol]["Item " + str(item)]
            if u_li.count() == 0:  # Checks if the result is empty
                continue
            else:
                u_li = u_li.iloc[0]
            '''

            nt = (fpt[vol]*lxk[vol,k]*ak[k])/pl[vol]
            rt = nt*u_li
            rik[item,k] = rik[item,k] + rt
            fik[item,k] = fik[item,k] + nt

        # M-step: Bayes modal estimation of item parameters
        alpha = np.zeros(n_questions)

        #for i in range(n_questions):
        for nit in range(n_questions + 1):

            alpha_i = math.log(a_i)

            L1 = 0
            L2 = 0
            L3 = 0

            A11 = 0
            A22 = 0
            A33 = 0

            A12 = 0
            A13 = 0
            A23 = 0

            # Theta loop
            for k in range(10):
                if fik[item,k] == 0:
                    continue

                # Cálculo de P*(x), Q*(x), P(x) e Q(x)
                dev = - a_i*(x[k] - b_i)
                p_star = 1/(1 + math.exp(dev))
                q_star = 1 - p_star

                p = c_i + (1 - c_i)*p_star
                q = 1 - p

                # Wik calculation for three parameters
                if p*q > 0.0000009:
                    w =  (p_star*q_star) / (p*q)
                else:
                    continue

                if (w < 0.0000009) or (w > 1e10):
                    continue

                rmf = rik[item,k] - fik[item,k]*p
                xmb = x[k] - b_i
                xmb2 = xmb**2

                pmc = p - c_i
                umc = 1 - c_i
                umc2 = umc**2
                pmc2 = pmc**2

                # -------------------------------------- #
                # Quadrature sum increments #
                # -------------------------------------- #
                # Likelihood of the first derivative with respect to parameters a, b and c
                L1 += rmf*xmb*w
                L2 += rmf*w
                L3 += rmf/p

                # The expectation of the second and cross derivatives
                A11 += fik[item,k] * xmb2 * (pmc2/umc2) * (q/p)
                A22 += fik[item,k] * (pmc2/umc2) * (q/p)
                A33 += (fik[item,k]/umc2) * (q/p)

                A12 += fik[item,k] * xmb * (pmc2/umc2) * (q/p)
                A13 += fik[item,k] * xmb * (pmc/umc2) * (q/p)
                A23 += fik[item,k] * (pmc/umc2) * (q/p)

            # Calculates the exponential of alpha
            ea = math.exp(alpha_i)
            ea2 = ea**2

            # Multiply by EA and ADD prior terms
            L1 = ea*umc*L1 - (alpha_i - pm_alpha)/pv_alpha
            L2 = - ea*umc*L2 - (b_i - pm_b)/pv_b
            L3 = L3/umc + (alp_c - 2)/c_i - (bet_c - 2)/umc

            A11 = - ea2*A11 - 1/pv_alpha
            A22 = - ea2*A22 - 1/pv_b
            A33 = - A33 - ((alp_c - 2)/(c_i**2)) - ((bet_c - 2)/umc2)

            A12 = ea2*A12
            A13 = - ea2*A13
            A23 = ea*A23

            # Calculation of the determinant of the 3x3 matrix of A's:
            #  A11 A12 A13
            #  A21 A22 A23
            #  A31 A32 A33
            # with A12 = A21; A13 = A31; A23 = A32
            det = A11*A22*A33 + 2*A12*A13*A23 - (A11*(A23**2) + A22*(A13**2) + A33*(A12**2))

            if abs(det) <= 0.000099:
                print("Out of bounds error")
                print('Determinante = {}'.format(det))
                a_i = math.exp(alpha_i)
                break

            # psi(t+1) = psi(t) - [Aij]^(-1) * [L]
            # To calculate the inverse of [Aij], we calculate its determinant and the cofactor matrix:
            C11 = A22*A33 - A23**2
            C22 = A11*A33 - A13**2
            C33 = A11*A22 - A12**2

            C12 = A13*A23 - A12*A33
            C13 = A12*A23 - A13*A22
            C23 = A12*A13 - A23*A11

            # Increment calculation for each parameter a, b and c
            da = (C11*L1 + C12*L2 + C13*L3)/det
            db = (C12*L1 + C22*L2 + C23*L3)/det
            dc = (C13*L1 + C23*L2 + C33*L3)/det

            alpha_i = alpha_i - da
            b_i = b_i - db
            c_i = c_i - dc
            a_i = math.exp(alpha_i)

            if abs(alpha_i) > 30 or abs(b_i) > 20 or abs(c_i) > 20:
                print("Out of bounds error")
                alpha_i = 0.0
                b_i = 0.0
                c_i = 0.2
                a_i = math.exp(alpha_i)
                break

            if abs(da) <= 0.05 and abs(db) <= 0.05 and abs(dc) <= 0.05:
                #print("Convergence criterion achieved")
                break

    return [pl, lxk, rik, fik, a_i, b_i, c_i]







if __name__ == '__main__':
    process = 4  # Define the Process
    level = 2  # Levels to define n(t) and m(v)
    n_episodes = 100   # Number of episodes

    sheet_name = 'test_unknown_tasks_qtd_solucoes_uu_' + test_name[test_type] + '_' + str(process) + '_' + str(level)
    sheet_name_ass = 'test_unknown_tasks_assignments_uu_' + test_name[test_type] + '_' + str(process) + '_' + str(level)
    sheet_name_tasks_param = 'test_unknown_tasks_parameters_uu_' + test_name[test_type] + '_' + str(process) + '_' + str(level)

    # Tries and solutions
    dict_tries = {'1': 16, '2': 14, '3': 19, '4': 17}
    if level == 0 or level == 1 or level == 2:
        n_tries = np.array(10000 * [dict_tries[test_type]])  # Levels 0, 1 and 2
    else:  # level == 3 or level == 4:
        n_tries = n_solved  # Levels 3 and 4

    # Tries and solutions
    n_volunteers = 100
    n_questions = 45



    '''
    # Getting the items parameters
    a = np.array(df_param[df_param['Type'] == test_type]['Dscrmn'])
    b = np.array(df_param[df_param['Type'] == test_type]['Dffclt'])
    c = np.array(df_param[df_param['Type'] == test_type]['Gussng'])
    '''



    # Simulate for many episodes
    qtd_total_solucoes = []
    #qtd_total_presented = 0

    for ep in range(94, n_episodes):
        print('Iniciando episódio ',ep)

        pl = np.zeros(n_volunteers)
        lxk = np.ones((n_volunteers, 10))


        # Initialization of item parameters to be estimated
        b = np.zeros(n_questions)
        a = np.ones(n_questions)
        c = []
        for i in range(n_questions):
            c.append(0.2)

        # r bar and f bar loop
        rik = np.zeros((n_questions, 10))
        fik = np.zeros((n_questions, 10))  # it is the "nk"


        # Dataframe with resource responses from the current episode
        df_resp_episode = df_resp.iloc[sampled_volunteers[(n_volunteers * ep):(n_volunteers * (ep + 1))]]

        s = list(df_resp_episode.sum(axis=1))  # FREQUENCY OF RAW SCORES FOR 1 TO MAXSCORE
        fdg = list(df_resp_episode.sum(axis=0))  # ITEM SCORE FREQUENCY

        # Defines the dataframe that will contain the information for the assignment excel
        df_new_ep = pd.DataFrame()

        # Defines the dataframe that will contain information for Excel about task parameters
        df_new_ep_tasks_param = pd.DataFrame()

        # Initializes the time count
        start = time.time()

        # Authenticate to google spreadsheet
        auth.authenticate_user()
        #gc = gspread.authorize(GoogleCredentials.get_application_default())  # This token only lasts for one hour (you will only write one episode at a time)
        creds, _ = default()
        gc = gspread.authorize(creds)

        #gc.login()  # Refreshes the token
        # Creation of the spreadsheet
        if ep == 0:
            sh = gc.create(sheet_name)
            sh_ass = gc.create(sheet_name_ass)
            sh_task_param = gc.create(sheet_name_tasks_param)

        # Open our new sheet and add some data.
        worksheet = gc.open(sheet_name).sheet1

        gs_ass = gc.open(sheet_name_ass)
        worksheet_ass = gs_ass.sheet1

        gs_tasks_param = gc.open(sheet_name_tasks_param)
        worksheet_tasks_param = gs_tasks_param.sheet1




        # Load the parameters to solve the LP
        n_tries_100 = n_tries[sampled_volunteers[(n_volunteers * ep):(n_volunteers * (ep + 1))]]
        theta_100 = np.random.normal(0, 1, n_volunteers)        # unknown thetas (KU or UU)
        # theta_100 = all_theta[sampled_volunteers[(n_volunteers * ep):(n_volunteers * (ep + 1))]]    # All known thetas (KK)

        # Calculation of parameter k for each group of 100 sampled volunteers
        perc_resp_correta = list()

        for i in range(n_questions):
            perc_resp_correta.append(df_resp.iloc[sampled_volunteers[(n_volunteers * ep):(n_volunteers * (ep + 1))], i].mean())

        dict_sol = {'1': [12, 35], '2': [9, 29], '3': [17, 41], '4': [17, 41]}
        if level == 0:
            n_solutions = n_questions * [dict_sol[test_type][0]]  # Level 0
        elif level == 1:
            n_solutions = n_questions * [dict_sol[test_type][1]]  # Level 1
        elif level == 2 or level == 3:
            n_solutions = np.array(perc_resp_correta) * n_volunteers  # Levels 2 and 3
        else:  # level == 4:
            n_solutions = np.array(perc_resp_correta) * n_volunteers * 0.5  # Level 4

        # Number of times each question was solved
        vezes_resolvidas = np.zeros(n_questions)
        qtd_total_presented = 0

        # Tasks presented for each volunteer
        tarefas_apresentadas = dict()
        tentativas = dict()
        respostas_apresentadas = dict()

        # Matrix with the probabilities and the answers for each item
        num = 1000
        probs = np.zeros((n_questions, num))

        if process == 2:
            # Solving the LP
            df_s = linear_programming_mod(n_questions, n_volunteers, a, b, c, theta_100, n_tries_100, n_solutions, vezes_resolvidas, ep)

        # Each volunteer is assigned a task and the probability of the user solving it is calculated
        for v_ in range(n_volunteers):
            df_new_resource = pd.DataFrame()

            if process == 3 or process == 4:
                v = (n_volunteers - 1) - v_
            else:
                v = v_

            # Each volunteer receives up to m different tasks
            tentativas[v] = 0
            tarefas_apresentadas[v] = list()
            respostas_apresentadas[v] = list()

            # Get the vector with the enem answers for volunteer v in episode ep
            resp = np.array(df_resp.iloc[sampled_volunteers[v + n_volunteers * ep]])

            '''
            if process == 3:
                # Estimate the theta of the current volunteer v
                resp = np.array(df_resp.iloc[sampled_volunteers[v + n_volunteers * ep]])
                theta_eap, _ = te.expected_a_posteriori(n_questions, a, b, c, resp)
                theta_100[v] = theta_eap
                # Solve the LP for each volunteer
                df_s = lp.linear_programming_mod(n_questions, v + 1, a, b, c, theta_100, n_tries_100, n_solutions, vezes_resolvidas, ep)
            '''
            while tentativas[v] < n_tries[sampled_volunteers[v + n_volunteers * ep]]:
                # Obtained the task to be presented to the volunteer
                if process == 2 or process == 3:
                    t = pl_policy(v, df_s)
                elif process == 4:
                    # Estimate the theta for volunteer v with each submission\response

                    if tentativas[v] == 0:
                        t = random_task(v)

                    # Estimation of task parameters
                    pl, lxk, rik, fik, a[t], b[t], c[t] = bayesian_item_parameter_estimation(pl, lxk, rik, fik, resp[t], t, v, a[t], b[t], c[t])

                    # Estimation of "theta" ability
                    theta_eap, _ = expected_a_posteriori_mod(t, a[t], b[t], c[t], respostas_apresentadas[v], probs, num, tarefas_apresentadas[v])
                    theta_100[v] = theta_eap


                    # Solve the LP for each submission (item presented to a volunteer)
                    df_s = linear_programming_mod(n_questions, v + 1, a, b, c, theta_100, n_tries_100, n_solutions, vezes_resolvidas, ep)
                    t = pl_policy(v, df_s)  # Presents a task and depending on the answer, the theta estimate will be updated and the next question to be presented will be updated

                else:
                    t = -1

                if t == -1:
                    break
                else:
                    # Verification by enem data if the student solved the task
                    # resp = np.array(df_resp.iloc[sampled_volunteers[v + n_volunteers * ep]])
                    resolvida = resp[t]  # = u_li

                    # Updates the number of times that task has been solved and increases attempts
                    vezes_resolvidas[t] += resolvida
                    tentativas[v] += 1
                    tarefas_apresentadas[v].append(t)
                    respostas_apresentadas[v].append(resp[t])

            qtd_total_presented += tentativas[v]

            df_new_resource['Episódio'] = [ep]*len(tarefas_apresentadas[v])
            df_new_resource['Recurso']  = [v]*len(tarefas_apresentadas[v])
            df_new_resource['Tarefas apresentadas'] = tarefas_apresentadas[v]
            df_new_resource['Respostas apresentadas'] = respostas_apresentadas[v]

            df_new_ep = df_new_ep.append(df_new_resource)


            df_new_ep_tasks_param['Episódio'] = [ep]*n_questions
            df_new_ep_tasks_param['Tarefa'] = list(range(n_questions))
            df_new_ep_tasks_param['a'] = a
            df_new_ep_tasks_param['b'] = b
            df_new_ep_tasks_param['c'] = c


        qtd_total_solucoes.append(vezes_resolvidas.sum())

        # Calculate the time at the and of the episode
        end = time.time()

        # Save the number of solutions at the end of each episode
        cells = worksheet.range('A' + str(ep+1) + ':G' + str(ep+1))
        cells[0].value = ep
        cells[1].value = vezes_resolvidas.sum()
        cells[2].value = end - start
        cells[3].value = qtd_total_presented
        worksheet.update_cells(cells)

        if ep == 0:
            gd.set_with_dataframe(worksheet=worksheet_ass, dataframe=df_new_ep, include_index=False, include_column_header=False)

            gd.set_with_dataframe(worksheet=worksheet_tasks_param, dataframe=df_new_ep_tasks_param, include_index=False, include_column_header=False)

        else:
            df_values = df_new_ep.values.tolist()
            gs_ass.values_append('Página1', {'valueInputOption': 'RAW'}, {'values': df_values})

            df_values_ = df_new_ep_tasks_param.values.tolist()
            gs_tasks_param.values_append('Página1', {'valueInputOption': 'RAW'}, {'values': df_values_})


    qtd_total_solucoes = np.array(qtd_total_solucoes)


    # Printing the results
    print('\nProcess: {0}, Level: {1}'.format(process, level))
    print("Mean of solutions for simulating {0} times: {1}".format(n_episodes, np.mean(qtd_total_solucoes)))
    print("Standard deviation of solutions for simulating {0} times: {1}".format(n_episodes, np.std(qtd_total_solucoes)))
    #print("Mean of tries for simulating {0} times: {1}".format(n_episodes, qtd_total_presented / n_episodes))



Iniciando episódio  94
